# DCGAN Workspace

### Discriminator Try

In [8]:
from numpy import expand_dims, ones, zeros
from numpy.random import rand, randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU

First, we need to build a discriminator model. This model will take a 28X28x1 image and down sample the image to 14x14 and then to 7x7. The reason for the extra dimension is to give a space for the value of the pixel (white or black, but in a sigmoid 0 to 1 format)

In [9]:
def create_discriminator(in_shape=(28,28,1)):
    """
    Creates a discrimator model
    
    Input:
    in-shape: This is the shape of the photos that will be put into the discriminator model.
    
    Output:
    The model for discriminating fake vs real images
    """
    model =  Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

Load mnist training images and generate fake images to train discriminator

In [10]:
# Load mnist data set
def load_mnist():
    """
    This function loads the MNIST dataset, and scales it to be in a sigmoid (0, 1) range
    """
    (trainX, _), (_, _) = load_data()
    # add third dimension for color value
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X / 255.0
    return X
 
# Randomly select real samples from MNIST to train with
def select_real_samples(dataset, n_samples):
    """
    Selects some number of real samples to train with from the input dataset.
    Uses random to select random images. Labels the images as 'real' with label = 1
    
    Inputs:
    dataset: Input dataset(MNIST)
    n_sample: number of samples to select
    
    Return:
    X: Selected images
    y: tags for images
    """
    # get n number of random images from dataset
    i = randint(0, dataset.shape[0], n_samples)
    
    X = dataset[i]
    y = ones((n_samples, 1))
    
    return X, y
 
# Create n amount of fake images to train discriminator
def create_fake_samples(n_samples):
    """
    Creates fake samples to train discriminator with correct dimensions
    
    Input:
    n_samples: number of samples to create
    
    Return:
    X: fake images
    y: image tags for training, 0 to mean not real images
    """
    X = rand(28 * 28 * n_samples)
    
    # Reshape and add tags to show fake
    X = X.reshape((n_samples, 28, 28, 1))
    y = zeros((n_samples, 1))
    return X, y

Train Discriminator Model

In [15]:
def train_discriminator(model, dataset, iterations=100, batch_size=256):
    """
    Trains the discriminator using mnist dataset and fake images.
    Takes half batch size of real and fake for each iteration.
    
    Inputs:
    model: input model
    dataset: loaded dataset (MNIST)
    iterations: number of iterations of training
    batch_size: images to train with in each iteration
    """

    for i in range(iterations):
        # Select real images and train discriminator
        X_real, y_real = select_real_samples(dataset, int(batch_size / 2))
        _, real_acc = model.train_on_batch(X_real, y_real)
        
        # Select fake images and train discriminator
        X_fake, y_fake = create_fake_samples(int(batch_size / 2))
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        
        #Performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))
 
model = create_discriminator()
dataset = load_mnist()
train_discriminator(model, dataset)

>1 real=39% fake=47%
>2 real=42% fake=64%
>3 real=47% fake=73%
>4 real=52% fake=88%
>5 real=49% fake=91%
>6 real=50% fake=98%
>7 real=61% fake=98%
>8 real=61% fake=100%
>9 real=61% fake=100%
>10 real=66% fake=100%
>11 real=73% fake=100%
>12 real=67% fake=100%
>13 real=77% fake=100%
>14 real=74% fake=100%
>15 real=75% fake=100%
>16 real=77% fake=100%
>17 real=82% fake=100%
>18 real=88% fake=100%
>19 real=81% fake=100%
>20 real=88% fake=100%
>21 real=91% fake=100%
>22 real=89% fake=100%
>23 real=95% fake=100%
>24 real=91% fake=100%
>25 real=92% fake=100%
>26 real=95% fake=100%
>27 real=95% fake=100%
>28 real=99% fake=100%
>29 real=98% fake=100%
>30 real=99% fake=100%
>31 real=100% fake=100%
>32 real=99% fake=100%
>33 real=98% fake=100%
>34 real=99% fake=100%
>35 real=100% fake=100%
>36 real=100% fake=100%
>37 real=100% fake=100%
>38 real=99% fake=100%
>39 real=98% fake=100%
>40 real=100% fake=100%
>41 real=100% fake=100%
>42 real=100% fake=100%
>43 real=100% fake=100%
>44 real=100% fake=

We can see above that with fake images as messy as randomly generated images along with real images from the mnist data set, we didn't need that many iterations or very big batch size to create a discriminator which could detect fakes pretty well.

In [17]:
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers.legacy import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# Suppress TensorFlow logs
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(learning_rate=0.00005, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
 
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.00005, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model
 
# load and prepare mnist training images
def load_real_samples():
    # load mnist dataset
    (trainX, _), (_, _) = load_data()
    # expand to 3d, e.g. add channels dimension
    X = expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
        # save plot to file
        filename = 'generated_plot_e%03d.png' % (epoch+1)
        pyplot.savefig(filename)
        pyplot.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
            # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
 
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

4/4 [==============================] - 0s 3ms/step
>1, 1/234, d=0.686, g=0.717
4/4 [==============================] - 0s 3ms/step
>1, 2/234, d=0.684, g=0.724
4/4 [==============================] - 0s 3ms/step
>1, 3/234, d=0.682, g=0.728
4/4 [==============================] - 0s 3ms/step
>1, 4/234, d=0.680, g=0.732
4/4 [==============================] - 0s 3ms/step
>1, 5/234, d=0.678, g=0.736
4/4 [==============================] - 0s 3ms/step
>1, 6/234, d=0.675, g=0.740
4/4 [==============================] - 0s 3ms/step
>1, 7/234, d=0.675, g=0.745
4/4 [==============================] - 0s 2ms/step
>1, 8/234, d=0.675, g=0.749
4/4 [==============================] - 0s 2ms/step
>1, 9/234, d=0.672, g=0.753
4/4 [==============================] - 0s 3ms/step
>1, 10/234, d=0.671, g=0.756
4/4 [==============================] - 0s 3ms/step
>1, 11/234, d=0.669, g=0.760
4/4 [==============================] - 0s 3ms/step
>1, 12/234, d=0.667, g=0.763
4/4 [==============================] - 0s 3ms/ste

KeyboardInterrupt: 

In [16]:
import tensorflow as tf
import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.15.0
Keras version: 2.15.0
